<div id="container" style="position:relative;">
<div style="float:left"><h1> Data Preprocessing and Modeling</h1></div>
<div style="position:relative; float:right"><img style="height:65px" src ="https://drive.google.com/uc?export=view&id=1EnB0x-fdqMp6I5iMoEBBEuxB_s7AmE2k" />
</div>
</div>

### Importing the data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re

In [2]:
# Read in the data and refresh our memories
df = pd.read_csv('Gluten-free1.csv')

In [3]:
# Check
df.head(1)

,ingredients,gluten-free?
0,"ALL PURPOSE GF FLOUR (BROWN RICE FLOUR, POTATO...",yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 727 entries, 0 to 726
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ingredients   727 non-null    object
 1   gluten-free?  727 non-null    object
dtypes: object(2)
memory usage: 11.5+ KB


In [5]:
ing_row = df["ingredients"]
ing_row.head()

0    ALL PURPOSE GF FLOUR (BROWN RICE FLOUR, POTATO...
1    ALL PURPOSE GF FLOUR (BROWN RICE FLOUR, POTATO...
2    GLUTEN FREE FLOUR (TAPIOCA STARCH, BROWN RICE ...
3    GLUTEN FREE FLOUR (TAPIOCA, WHITE RICE, SORGHU...
4    GLUTEN FREE FLOUR (TAPIOCA STARCH, WHITE RICE ...
Name: ingredients, dtype: object

------

### Cleaning the Data

In [10]:
# Define a regular expression pattern to match '()', '[]', and ','
pattern = r'[(),\[\]:]'

# Splitting the strings by the defined pattern and storing individual ingredients in a list
individual_ingredients = []
for row in ing_row:
    individual_ingredients.extend(ing_row)

# Printing the individual ingredients list
print(individual_ingredients)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Removing white space

In [7]:
clean_data_stripped = [item.strip() for item in individual_ingredients]


In [8]:
clean_data_stripped

['ALL PURPOSE GF FLOUR (BROWN RICE FLOUR, POTATO FLOUR, TAPIOCA FLOUR, POTATO STARCH, WHITE RICE FLOUR, XANTHAN GUM), ORGANIC CANE SUGAR, VEGAN BUTTERY SPREAD [(OIL BLEND (PALM FRUIT, CANOLA, OLIVE), WATER, SALT, NATURAL FLAVOR, SUNFLOWER LECITHIN, LACTIC ACID, ANNATTO EXTRACT], ORGANIC BROWN SUGAR, VEGAN CHOCOLATE (ORGANIC SUGAR, CHOCOLATE, COCOA BUTTER, SOY LECITHIN, VANILLA BEAN), WATER, TAPIOCA SYRUP, LESS THAN 2% OF: PALM FRUIT OIL, PURE VANILLA EXTRACT (WATER, ALCOHOL 40%, VANILLA BEANS), CREAM OF TARTAR, SUNFLOWER LECITHIN, KOSHER SALT, BAKING SODA, GUAR GUM, KONJAC, XANTHAN GUM',
 'ALL PURPOSE GF FLOUR (BROWN RICE FLOUR, POTATO FLOUR, TAPIOCA FLOUR, POTATO STARCH, WHITE RICE FLOUR, XANTHAN GUM), ORGANIC CANE SUGAR, GLUTEN FREE OATS, VEGAN BUTTERY SPREAD [(OIL BLEND (PALM FRUIT, CANOLA, OLIVE), WATER, SALT, NATURAL FLAVOR, SUNFLOWER LECITHIN, LACTIC ACID, ANNATTO EXTRACT], ORGANIC BROWN SUGAR, WATER, TAPIOCA SYRUP, PALM FRUIT OIL, BAKING SODA, PURE VANILLA EXTRACT (WATER, VANILL

Removing white space

-----

In [ ]:
# finding out how many indredients are in the file
len(clean_data_stripped)

#### Count unique values in a list

In [ ]:
Counter(clean_data_stripped)

Adding the data back into the dataset

-----

In [ ]:
gf_df = pd.DataFrame(clean_data_stripped, columns=['ingredients'])
gf_df.head(6)

Process of removing duplicates

In [ ]:
gf_df.shape

In [ ]:
gf_df.duplicated().sum()

In [ ]:
# Check in terms of percentages
gf_df.duplicated().sum()/gf_df.shape[0]*100

In [ ]:
clean_data = gf_df.drop_duplicates()

In [ ]:
clean_data.shape

In [ ]:
clean_data.value_counts()

In [ ]:
clean_data_stripped = [item.strip() for item in clean_data]

In [ ]:
clean_data_stripped


-------

-----

# Adding New Dataset

This shows us that the target column is not balanced - far more customers are not registered for the product opposed to those who did (60/40 split). We will convert this column to a binary one, where 1 represents registering and 0 represents not registering. 

In [ ]:
df['registered'] = np.where(df['registered'] == 'yes', 1, 0)

In [ ]:
#Sanity Check
df.head()

In [ ]:
#Sanity Check 2

df['registered'].value_counts()

Now we've converted the target. Let's ensure it's the correct datatype and look at which other columns need changing. 

In [ ]:
df.info()

Next, let's look at categorical columns with fewer distinct values as they will be quicker to preprocess. Let's start with the `marital` column:

In [ ]:
#check 
df['marital']

Let's check the unique or distinct values in more detail.

In [ ]:
df['marital'].value_counts()

If we just had `married` or `divorced`, we could have converted this into a binary. However we have the `single` option as well, and this is a large chunk of the data as well. Much like `jobs` which we can tackle later. 

In [ ]:
df.head()

Looking at the above, it appears that `credit_in_default`, `housing_loan` and `personal_loan` are binary in nature. Let's examine these and see if we can convert them such that that can be read in a model.

In [ ]:
#Check
df['credit_in_default'].value_counts()

Looking at the above, there are a lot of customers with unknown credit status but only 1 with 'yes'. Some of the unknown are likely in default. We make the argument that these are different things from a data standpoint (in terms of what the bank knows about the customer). Therefore , we will consider credit not in defualt to be zero, otherwise (unknnonw and yes) as 1.

In [ ]:
# Here because there are multiple values we will use a map function

df['credit_in_default'].map({'no': 0, 'yes': 1, 'unknown': 1})

In [ ]:
df['credit_in_default'] = df['credit_in_default'].map({'no': 0, 'yes': 1, 'unknown': 1})

In [ ]:
#Check
df['credit_in_default'].value_counts()

We have binarized credit in defualt. Let's change the name of the column based on our assumption that there is a higher likelihood of unknowns being actual credit_in_default than not. 

In [ ]:
#Rename our column
df.rename(columns={'credit_in_default':'known_credit_in_default'}, inplace=True)

In [ ]:
df.head()

What else is left.

In [ ]:
df.info()

Hopefully Housing and Personal Loans will require a similar approach. Let's take a look:

In [ ]:
df['housing_loan'].value_counts()

In [ ]:
df['personal_loan'].value_counts()

It looks like 260 people have unknown in each column. In this case we can check if it's the same people. 

In [ ]:
df[(df['housing_loan'] == 'unknown') & (df['personal_loan'] == 'unknown')]

Seems like it's the same people who might have left that column blank. Let's plot the distributions as well.

In [ ]:
plt.subplots(1,2, figsize=(8, 6))

plt.subplot(1,2,1)
df['personal_loan'].value_counts().plot(kind='barh')
plt.title("Personal Loans")
plt.xlabel("Counts")


plt.subplot(1,2,2)
df['housing_loan'].value_counts().plot(kind='barh')
plt.title("Housing Loans")
plt.xlabel("Counts")

plt.tight_layout()

plt.show()

Looking at these we can give it the same treatment as credit_in_default. Therefore we will lump the unknowns to `no`. and rename the column.

In [ ]:
#Personal Loan
df['known_personal_loan'] = df['personal_loan'].map({'no': 0, 'unknown': 0, 'yes': 1})

In [ ]:
#Sanity Check
df['known_personal_loan'].value_counts()

Let's do the same thing with housing_loans.

In [ ]:
#Personal Loan
df['known_housing_loan'] = df['housing_loan'].map({'no': 0, 'unknown': 0, 'yes': 1})

In [ ]:
#Sanity Check
df['known_housing_loan'].value_counts()

Let's drop the original columns for these 2.

In [ ]:
df.drop(columns=['housing_loan', 'personal_loan'], inplace=True)

Sanity check plus what else is left.

In [ ]:
df.info()

Let's look at last_contact_type.

In [ ]:
df['last_contact_type'].value_counts()

Lucky for us, this is 2 distinct types. Let's encode it as `last_contact_type_cellular`

In [ ]:
df['last_contact_type_cellular'] = np.where(df['last_contact_type'] == 'cellular', 1, 0)

In [ ]:
#Check
df['last_contact_type_cellular'].value_counts()

We will drop last_contact_type

In [ ]:
df.drop(columns='last_contact_type', inplace=True)

In [ ]:
df.info()

In [ ]:
for col in ['location', 'prev_campaign_outcome', 'last_contact_day', 'last_contact_month', 'education', 'marital', 'job']:
    print(col.upper()) #Upper() makes it uppercase
    print(df[col].value_counts())
    print('\n')
    print("-------------------------------")

---

As seen above, we need to deal with columns that have multiple options. Let's start with Education. We can group all basic education as 1 and illiterate and unknown.

In [ ]:
df['education'] = df['education'].map({'university.degree': 'university.degree',
                                      'unknown': 'unknown',
                                      'high.school': 'high.school',
                                      'professional.course': 'professional.course',
                                      'illiterate': 'unknown',
                                      'basic.6y': 'basic',
                                      'basic.4y': 'basic',
                                      'basic.9y': 'basic'})

In [ ]:
#Sanity Check
df['education'].value_counts()

Let's also look at the date and day columns in the data to see if they have any trends. Potentially customers may be more likely to sign up on one day compared to another (similar to months). We can also see seasonality patterns (this is a hypothesis). We will encode this with increasing integers for our model.


In [ ]:
#last_contact_day
df['last_contact_day'].value_counts()

We see there are no contacts on the weekend. If we had them we could have done a binary of week vs weekend. 

In [ ]:
df['last_contact_day'] = df['last_contact_day'].map({'mon': 1, 'tue':2, 'wed': 3, 'thu': 4, 'fri': 5})

In [ ]:
df['last_contact_day'].value_counts()

In [ ]:
df['last_contact_day'].value_counts().sort_index().plot(kind='bar')
plt.title("Count of Customers being contacted by weekday")
plt.xlabel("Weekday #")
plt.ylabel("Count of contacts")
plt.show()

Interesting to note, appears to be more contacts on Thursday, and less on Fridays than on other days of the week.

In [ ]:
# Last contact month
df['last_contact_month'].value_counts()

In [ ]:
# Try pd.to_datetime instead of using map...
df['last_contact_quarter'] = pd.to_datetime(df['last_contact_month'], format='%b').dt.quarter

For the months, we encode it based on the results of the previous analysis as 'good months' (Sep, Oct, Dec, Mar, Apr) and 'bad months' (everything else). The good months show a higher percent of people signing up.

In [ ]:
# alernative - code as good month bad month as 

good_months = ['apr', 'oct', 'mar', 'sep', 'dec']

df['good_month'] = 0

df.loc[df['last_contact_month'].isin(good_months), 'good_month'] = 1

In [ ]:
df['good_month'].value_counts()

Now we have rolled up the last contact month to the quarter and encoded these as integers. We choose to drop last contact month after this, although this a choice we are making - we could have a greater level of detail by encoding last contact month as 1,2,3,...12 if we had so chosen.

In [ ]:
df['last_contact_quarter'].value_counts().sort_index().plot(kind='bar')
plt.show(df['last_contact_quarter'].value_counts().sort_index().plot(kind='bar'))
plt.show()

In [ ]:
# Drop last_contact_month
df.drop('last_contact_month', axis=1, inplace=True)

Interesting to note above, very little contact in the data in Q1, most of the customers in the campaign were previously contacted in Q2, and then this drops steeply off up until Q4.

What's left?

In [ ]:
df.info()

#### Dummy Variables

To numerically represent catregorical variables with many nominal distinct values we can use ***dummy variables***. A dummy variable is a binary variable that takes values of 0 and 1, where the values indicate the presence or absence of something. A categorical variable that has more than two categories can be represented by a set of dummy variables, with one variable being used to indicate the presense/absence for each category. 

Dummy variables are also known as ***One-hot encoding*** since only one of the dummy variables for each category can have a value of 1 at a time. 

In [ ]:
states = pd.Series(['NY','Boston','Texas','NY'])
states


In [ ]:
pd.get_dummies(states)

In [ ]:
df.head()

In [ ]:
# Stating dummy variables for our categorical columns

df['job'].value_counts()

In [ ]:
df['marital'].value_counts()

In [ ]:
pd.get_dummies(df['marital'])

When you have a categorical column with a lot of classes, the first thing to do is to group them and make sure that dummy variables wouldn't lead to many columns (20-30). For this example, we will drop the location column but other options would be to group the locations based on the city names and reduce the number of dummy variables.

In [ ]:
df.drop('location', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.select_dtypes('object')

In [ ]:
# Dataframe with dummy variables
dummy_df = pd.get_dummies(df)

In [ ]:
# To see all of the columns
pd.set_option('display.max_columns',200)

In [ ]:
dummy_df.head()

In [ ]:
# Decision about dropping column

pd.get_dummies(df['marital']).var()

In [ ]:
pd.get_dummies(df['job']).var().sort_values(ascending=False)

In [ ]:
pd.get_dummies(df['job']).sum().sort_values(ascending=False)

In [ ]:
pd.get_dummies(df['prev_campaign_outcome']).sum().sort_values(ascending=False)

In [ ]:
# Columns to drop to avoid multicollinearity from our dummy variables
refrence_Categories = ['marital_unknown', 'job_unknown', 'education_unknown', 'prev_campaign_outcome_nonexistent'] 

In [ ]:
dummy_df.drop(columns=refrence_Categories, inplace=True)

In [ ]:
dummy_df.info()

In [ ]:
dummy_df.sample(10)

### Part 2: Modeling

Now that we are satisfied our data is ready to be put into a model, we proceed to the model building phase of our analysis. First, we need to ensure that certain assumption hold about the data: these include our samples being independent and identically distributed, and that the independent variables are not collinear or show multicollinearity.

In [ ]:
# Pull out features and target for modeling

X = dummy_df.drop('registered', axis=1)
y= dummy_df['registered']

#### Collinearity & Multicollinearity

**Applies to:**

   - Linear & Logistic regression models

**What is it?**

- **Collinearity** refers to the situation when two independent variables are correlated with one another.
- **Multicollinearity** is the situation where one independent variable can be expressed as a linear combination of two or more other independent variables (in other words, the independent variables are in a linear relationship with *each other*).

**What does it do to our models?**

 - We often see certain independent variables be given much lower coefficients than what the correlation between that variable and the dependent variable would suggest. It may also go against our prior subject matter knowledge.
 
 - The coefficient standard errors may also be very high, suggesting that the model is unsure about its prediction.
 
 - As a result of both of the above, p-values are likely to be highly skewed, and may lead us to make incorrect conclusions about the significance of the variable. 
 
**Why does this happen?**

Fundamentally, linear regression models use how an independent variable varies with a dependent variable to estimate the coefficient value that describes the change in the $y$. This is why correlation between the independent variable and dependent variables is important to have. If there is a strong relationship the model can look at the data and say "Ah, if $x$ increases by 1, then $y$ generally increases by ~2"

In multiple linear regression, where we have multiple independent variables, we also need to factor in how the independent variables vary with *each other* as well as the dependent variable. Consider this extreme scenario:

   - Two independent variables $x_1$ and $x_2$ have very strong correlation with each other, |$\rho|\geq 0.9 $
   - Both variables variables share strong positive correlation with the dependent $y$, $\rho \approx 0.7 $

In the scenario above, the model will be unsure about which independent variable is the one that actually driving the relationship with the dependent variable. Ultimately both independent variables seem to move in the same way. 

The result of this confusion is that the model will assign the majority of the relationship to one of the variables, and assign little to the other. As a result, one coefficient value will be high and the other very low! This may not match up with what we would expect and lead us to make conclusions about the relationship between the independent variables and the dependent variable that do not reflect the true relationship.

#### Detecting Collinearity

Collinearity can be detected by finding the pairwise **correlation** between the independent variables (we typically visualize this as a heatmap). If a linear/logistic regression model is to be fitted, this should always be part of the exploratory phase. If the correlation between some predictors is high, it is a sign of collinearity and we can consider including only one of those predictors in our models (or to combine the information in those predictors some way).

Let us explore collinearity in our dataset.

In [ ]:
X.head()

In [ ]:
# Calculated the correlation matrix

corr_df = X.corr()


mask = np.triu(corr_df)
plt.figure(figsize=(20,20))
sns.heatmap(corr_df.round(2), annot=True,mask=mask, vmin=-1, vmax=1, cmap='coolwarm')
plt.show()

In [ ]:
X[['prev_campaign_outcome_success','n_contacts_prev_campaign']].describe()

In [ ]:
X.drop('n_contacts_prev_campaign',axis=1 ,inplace=True)

In [ ]:
# Calculated the correlation matrix

corr_df = X.corr()


mask = np.triu(corr_df)
plt.figure(figsize=(20,20))
sns.heatmap(corr_df.round(2), annot=True,mask=mask, vmin=-1, vmax=1, cmap='coolwarm')
plt.show()

In [ ]:
X[['marital_married','marital_single']].describe()

In [ ]:
X.drop('marital_single',axis=1 ,inplace=True)

In [ ]:
# Calculated the correlation matrix

corr_df = X.corr()


mask = np.triu(corr_df)
plt.figure(figsize=(20,20))
sns.heatmap(corr_df.round(2), annot=True,mask=mask, vmin=-1, vmax=1, cmap='coolwarm')
plt.show()

#### Detecting Multicollinearity with the Variance Inflation Factors

Looking at correlations only won't detect multicollinearity, hence we need a new tool: the **Variance Inflation Factor** (VIF). 

In order to calculate it, we build a regression model of each independent variable against the other independent variables and look at the $R^2$ score. The VIF for each predictor is defined as 
$$
\text{VIF}_i = \frac{1}{1-R_i^2}
$$

In a perfect scenario of no multicollinearity, the VIF for each predictor should be 1 (since the $R^2$ from each model would be 0, showing that the given independent variable can't be modeled by the other variables, hence no linear relationship). 

By common convention, any VIF value higher than 5 indicates high multicollinearity. Let's examine the VIF for the variables in our dataset:

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
X_withcons = sm.add_constant(X)

In [ ]:
X.head()

In [ ]:
variance_inflation_factor(X_withcons.values, 1)

In [ ]:
pd.Series([variance_inflation_factor(X_withcons.values, i) for i in range(X_withcons.shape[1])],
         index = X_withcons.columns)[1:]

A VIF score of above 5 (R2 >=0.8) indicates multi colliniearity. So in this case we don't need to be worried about that.

#### Variable Selection

You will hopefully find that some combinations of variables will give almost the same performance using fewer predictors and hence, less multicollinearity. 


Even if there isn't any collinearity or multicollinearity present, it's common to try and find a simpler model that will have almost the same performance as a more complicated model. This is usually done by adding or removing variables until all are significant (p-value < 0.05). 

The technical term for this is called variable selection. In fact there's a few way to do this, some of which are: 

- **Forward Selection**: Starting with the intercept only, at each step, select the candidate variable that increases R-Squared the most.

- **Backward Selection**: Starting with a full model (all predictors), at each step, the variable that is the least significant is removed. In the case of multicollinearity, the variable removed may be the one with the highest VIF scores. 

- **Stepwise Selection**: Combination of the above two, after each step in which a predictor was added, all predictor candidates in the model are checked to see if their significance has been reduced below a p-value of 0.05.


#### Forward Feature Selection

The following diagram demonstrates the forward selection approach:

<img src="https://drive.google.com/uc?export=view&id=19RfhvLdwjnlps_Kx4qMZWrpJQab_E2Hi" alt="Backward Selection" width="600" height="700"/>


#### Backward Feature Selection

The backward selection approach can be imagined as this:

<img src="https://drive.google.com/uc?export=view&id=1HbSW65W_tZR6N5wCFDTtsabYwDSAT3QP" alt="Backward Selection" width="600" height="700"/>

### Backward Selection
In backward selection, we fit a model with all data available to us to start, then remove variables based upon whether or not they are significant (and informed by subject matter knowledge), until will reach a model of desired simplicity with sufficient predictive power.

Let's start the feature selection process with our data set now:


In [ ]:
import statsmodels.api as sm

In [ ]:
# 0: Add the constant

X_cons = sm.add_constant(X)

#1: Instantiate the model

bank_logit = sm.Logit(y, X_cons)

#2: Fit the model

bank_logit_fitted = bank_logit.fit()

In [ ]:
bank_logit_fitted.summary()

We note above, that for many variables, the associated p-values with the coefficients (the betas) are large. Therefore, we cannot safely reject the null hypothesis ($\beta_i = 0$), so in pratical terms, these independent variables are likely not predictive of the dependent variable. We try excluding these from the model and seeing if this has an effect or not. Note that removing many variables at once is to be avoided, as the p-values will vary based upon which are left in or out of the model together in combination.

Regardless of this fact, we can still make a prediction for y. We will do this leaving in the insignificant variables to establish a baseline accuracy:

### Part 4: Model Evaluation & Summary

We've made some models, now let's try to interpret them and summarize our findings.

In [ ]:
# Making soft prediction
y_prob = bank_logit_fitted.predict(X_cons)

y_prob

In [ ]:
y_pred = np.where(y_prob>=0.5, 1, 0)

y_pred

Let's now calculate the model accuracy by comparing the predictions with the actual values

In [ ]:
correct_predictions = (y_pred == y).sum()

total_obs = X_cons.shape[0]

# The model accuracy

print(f'The model accuracy is {round(correct_predictions/total_obs,2)}')

## Iteration 2

Dropping columns with high p values to see how it affects the model performance.

In [ ]:
# 0: Add the constant

X2_cons = sm.add_constant(X)

#1: Instantiate the model

bank_logit2 = sm.Logit(y, X2_cons)

#2: Fit the model

bank_logit_fitted2 = bank_logit2.fit()

In [ ]:
bank_logit_fitted2.summary()

In [ ]:
# Making soft prediction
y_prob2 = bank_logit_fitted2.predict(X2_cons)

y_prob2

In [ ]:
# Hard prediction
y_pred2 = np.where(y_prob2>=0.5, 1, 0)

y_pred2

In [ ]:
correct_predictions2 = (y_pred2 == y).sum()

total_obs = X2_cons.shape[0]

# The model accuracy

print(f'The model accuracy is {round(correct_predictions2/total_obs,2)}')

## Model interpretation


In [ ]:
bank_logit_fitted2.params

In [ ]:
plt.figure(figsize=(10,10))
bank_logit_fitted2.params.sort_values(ascending=True).plot(kind='barh')
plt.show()

Looking at the figure above `prev_campaign_outcome_success` as is the month of contact and type of that contact. In terms of demographics, students or retired folks are more likely to sign up.

Factors that are negatively predictive, that is to say, mean a customer is less likely to accept to product are having their credit in default, and a blue collar or services role, to name a few.

In [ ]:
highest_param = bank_logit_fitted2.params.sort_values(ascending=False)['prev_campaign_outcome_success']

To quantify further, we could look at the log odds. For example, for the student variable:


In [ ]:
np.exp(highest_param)

Having a `prev_campaign_outcome_success` can increase the odds of someone getting the loan increases by 15. 

To take this a step even further, we could calculate the log odds for variables of interest and plot this information. This would give us and our stakeholders a handy reference when discussing variable importance. 

In [ ]:
coefficient_df = pd.DataFrame({'coef': bank_logit_fitted2.params, 
                              'pvals': bank_logit_fitted2.pvalues})



In [ ]:
coefficient_df

In [ ]:
coefficient_df.reset_index(inplace=True)
coefficient_df.rename({'index':'variable'}, axis=1, inplace=True)

In [ ]:
coefficient_df.head()

In the above cell, we can filter out the variables with pvals less than 5% and then use them to run a model and compare its performance with the previous one that we built.

In [ ]:
coefficient_df[coefficient_df['pvals']<0.05]['variable']

Let's build a bar chart based on this data. We will convert coefficients into odds increases and decreases, color them in red and blue based on increase or decrease, and change the saturation based on p-values. 

In [ ]:
categories = []    # Holds labels for bars
sizes = []         # Hold bar heights
colors = []        # Holds bar colors
p_values = []      # Used for additional text within the plot

p_value_color_scale = 1.5

# iterate through
for index, row in coefficient_df.iterrows():
    variable_name = row['variable'].strip()
    
    if (variable_name != 'const'):   # We don't care much for the constant since we can modify it
        
        
        categories.append(variable_name)
        coefficient_value = row['coef']
        
        p_value = round(row['pvals'],2)
        p_values.append(p_value)
        


        color = [1.0,1.0,1.0]
        
        color[1] = (min(p_value_color_scale*p_value, 1.0)/1.0)  # Make GREEN brighter
        
        if (coefficient_value >= 0):
            color[0] = color[1]                     # Make RED brighter
            column_size = np.exp(coefficient_value)
        else:
            color[2] = color[1]                     # Make BLUE brighter
            column_size = -1/np.exp(coefficient_value)

        column_size = round(column_size,2)
        sizes.append(column_size)
        colors.append(color)

Let's create a bar chart with some text labels as well. If the p-value is larger than, or equal to, 0.05, we will label that bar

In [ ]:
fix, ax = plt.subplots(figsize=(16, 12))
ax.barh(categories, sizes, color=colors)
for index, bar_size in enumerate(sizes):
    if (bar_size > 0.0):
        # Add a label text
        ax.text(bar_size + 0.5, index-0.15, f'{bar_size} odds increase', color='blue')
        
        # Add a p-value disclaimer
        if (p_values[index] >= 0.05):
            ax.text(-2.5, index-0.15, f'p-value={p_values[index]}', color='blue')
    else:
        # Label text
        ax.text(bar_size - 3.7, index-0.15, f'{abs(bar_size)} odds decrease', color='red')
        
        # p-value disclaimer
        if (p_values[index] >= 0.05):
            ax.text(0.3, index - 0.15, f'p-value={p_values[index]}', color='red')

# change the limit to make sure labels go inside the plot area
plt.xlim(-7.5, 16.5)
plt.title("Effect of Different Variables on Odds Increase (blue) or Decrease (red).\
\nColors are Less Intense for Statistically Insignificant Items", size=20)
plt.xlabel("Odds change for a given feature", size=20)
plt.ylabel("Variable of interest", size=20)
plt.show()

#### Forward Selection vs. Backward

In forward selection, we will instead start with a single variable we think is most predictive (start with the simplest model) and then add variables instead of removing them as in backward selection above.

In [ ]:
# Look at correlation between each variable and the target
dummy_df.corr()['registered'].sort_values(ascending=False)

`last_contact_duration` appears to have the highest correlation with the target (`registered`), so we will start with it as our first variable to build our initial model:

In [ ]:
# Instantiate the model with a SINGLE variable - we will choose last_contact_duration given the above
new_X = X['last_contact_duration']
new_X_const = sm.add_constant(new_X)

# Instantiate
forward_logit = sm.Logit(y, new_X_const)

# Fit
fitted_forward = forward_logit.fit()

# Summary
fitted_forward.summary()

We can see in the above that the p-value is very small, so we can safely reject the null hypothesis that $\beta_1=0$ and say there is evidence to support it being predictive of the target outcome.

In [ ]:
# Calculate soft predictions
y_proba = fitted_forward.predict(new_X_const)

# Convert soft predictions to hard predictions 0/1
y_pred = np.where(y_proba >= 0.5, 1, 0)

# Calculate # correct
num_correct = (y_pred == y).sum()

# Calculate the percentage accuracy
pct_accuracy = num_correct / X.shape[0]

print(f'The baseline model accuracy is {pct_accuracy*100.0}%')

We were able to predict the target with 71% accuracy, using only a single variable! This is very close to our baseline performance in backward selection. The reality may be more complicated. We could do further analysis and continue to add variables and see the effect on model performance until reaching an optimal model with the best trade-off between predictive power and model complexity.

---

**Note:** continue adding features to this simple model and track how the coefficients and model performance changes.

---

<div id="container" style="position:relative;">
<div style="position:relative; float:right"><img style="height:25px""width: 50px" src ="https://drive.google.com/uc?export=view&id=14VoXUJftgptWtdNhtNYVm6cjVmEWpki1" />
</div>
</div>